<a href="https://colab.research.google.com/github/aya-rhouma/SmartTenderAINightChallenge/blob/main/SmartTenderAINightChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
from google.colab import files
uploaded = files.upload()

Saving AI_Resume_Screening.csv to AI_Resume_Screening (1).csv


In [81]:
!pip install gradio sentence-transformers pandas jinja2
import gradio as gr
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import jinja2
from datetime import datetime

In [82]:
df = pd.read_csv('AI_Resume_Screening.csv')
df['Certifications'] = df['Certifications'].fillna('')
for col in ['Skills', 'Education', 'Job Role', 'Certifications']:
    df[col] = df[col].astype(str).str.strip()

def build_full_text(row):
    return (f"{row['Job Role']}. Compétences: {row['Skills']}. "
            f"Expérience: {row['Experience (Years)']} ans. "
            f"Formation: {row['Education']}. "
            f"Certifications: {row['Certifications']}. "
            f"Projets: {row['Projects Count']}.")
df['full_text'] = df.apply(build_full_text, axis=1)

df_sample = df.head(200).copy()
profils = []
for idx, row in df_sample.iterrows():
    profils.append({
        "id": int(row['Resume_ID']),
        "nom": row['Name'],
        "titre": row['Job Role'],
        "competences": row['Skills'],
        "experience": f"{row['Experience (Years)']} ans",
        "texte_complet": row['full_text']
    })

print(f"{len(profils)} profils chargés.")

200 profils chargés.


In [83]:
model = SentenceTransformer('all-MiniLM-L6-v2')
profils_emb = model.encode([p["texte_complet"] for p in profils])
profils_emb_norm = profils_emb / np.linalg.norm(profils_emb, axis=1, keepdims=True)

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [84]:
# Fonction de matching
def matching(offre_text, top_k=3):
    offre_emb = model.encode([offre_text])[0]
    offre_emb_norm = offre_emb / np.linalg.norm(offre_emb)
    scores = np.dot(profils_emb_norm, offre_emb_norm)
    indices_tries = np.argsort(scores)[::-1]
    top_indices = indices_tries[:top_k]
    resultats = []
    for idx in top_indices:
        profil = profils[idx]
        mots_offre = set(offre_text.lower().split())
        mots_profil = set(profil["competences"].lower().replace(',', '').split())
        communs = mots_offre.intersection(mots_profil)
        communs_str = ", ".join(communs) if communs else "Aucune"
        resultats.append({
            "profil": profil,
            "score": float(scores[idx]),
            "communs": communs_str
        })
    return resultats

In [85]:
# Templates de réponses
template_acceptation = """
{{ entreprise }}
{{ adresse_entreprise }}
{{ date }}

Objet : Candidature pour le poste de {{ titre_offre }}

Madame, Monsieur {{ nom_candidat }},

Nous avons le plaisir de vous informer que votre candidature pour le poste de {{ titre_offre }} a été retenue.

{{ message_personnalise }}

Nous vous contacterons très prochainement pour organiser un entretien.

Dans l'attente de vous rencontrer, veuillez agréer, Madame, Monsieur, l'expression de nos salutations distinguées.

{{ nom_recruteur }}
{{ poste_recruteur }}
"""

template_refus = """
{{ entreprise }}
{{ adresse_entreprise }}
{{ date }}

Objet : Candidature pour le poste de {{ titre_offre }}

Madame, Monsieur {{ nom_candidat }},

Nous vous remercions d'avoir porté votre candidature au poste de {{ titre_offre }}.

Nous avons été sensibles à votre parcours et à vos compétences. Cependant, nous avons reçu de nombreuses candidatures de grande qualité et avons choisi un profil correspondant davantage à nos attentes spécifiques.

{{ message_personnalise }}

Nous vous souhaitons bonne chance dans vos recherches et restons à votre disposition pour toute information complémentaire.

Cordialement,
{{ nom_recruteur }}
{{ poste_recruteur }}
"""


In [86]:
def generer_reponse(profil, type_reponse, titre_offre, message_perso="",
                   entreprise="Inetum Tunisie",
                   adresse="Immeuble Inetum, Tunis",
                   recruteur="Service RH",
                   poste_recruteur="Responsable recrutement"):
    env = jinja2.Environment()
    template = env.from_string(template_acceptation if type_reponse == "Accepter" else template_refus)
    return template.render(
        entreprise=entreprise,
        adresse_entreprise=adresse,
        date=datetime.now().strftime("%d/%m/%Y"),
        titre_offre=titre_offre,
        nom_candidat=profil["nom"],
        message_personnalise=message_perso,
        nom_recruteur=recruteur,
        poste_recruteur=poste_recruteur
    )

In [87]:
with gr.Blocks(theme=gr.themes.Soft(), title="SmartTender AI - Matching & Réponses") as demo:
    gr.Markdown("""
    # 📄 SmartTender AI – Automatisation des réponses aux candidatures
    **Étape 1 :** Entrez une offre pour trouver les meilleurs profils.
    **Étape 2 :** Pour chaque profil, choisissez **Accepter** ou **Refuser**.
    **Étape 3 :** Cliquez sur "Générer les réponses" pour produire toutes les lettres.
    """)
    resultats_bruts = gr.State([])
    etat_choix = gr.State({})
    generated_letters_state = gr.State([])

    with gr.Row():
        offre_input = gr.Textbox(
            label="Description de l'offre",
            placeholder="Ex: Data Scientist avec expérience en Python, TensorFlow...",
            lines=3,
            scale=3
        )
        top_k_slider = gr.Slider(
            minimum=1, maximum=10, value=3, step=1,
            label="Nombre de résultats",
            scale=1
        )

    search_btn = gr.Button("Rechercher", variant="primary")

    titre_offre_input = gr.Textbox(
        label="Titre de l'offre (pour les lettres)",
        placeholder="Ex: Data Scientist",
        lines=1,
        visible=True
    )

    detected_roles_output = gr.Markdown("**Detected Job Role(s):** None")
    detected_skills_output = gr.Markdown("**Detected Skills:** None")

    with gr.Column() as results_container:
        @gr.render(inputs=[resultats_bruts, etat_choix])
        def render_cards(results, choix):
            if not results:
                gr.Markdown("Aucun résultat. Faites une recherche.")
                return

            for r in results:
                pid = r['profil']['id']
                with gr.Row(key=f"card_row_{pid}"):
                    with gr.Column(scale=1):
                        gr.Markdown(f"**{r['profil']['nom']}**")
                        gr.Markdown(f"*{r['profil']['titre']}*")
                    with gr.Column(scale=2):
                        gr.Markdown(f"**Score:** {r['score']:.3f}")
                        gr.Markdown(f"**Compétences communes:** {r['communs']}")
                    with gr.Column(scale=1):
                        current = choix.get(pid, None)
                        radio = gr.Radio(
                            ["Accepter", "Refuser"],
                            label="Décision",
                            value=current,
                            elem_id=f"radio_{pid}"
                        )
                        radio.change(
                            fn=lambda p_id, val, current_choices: {**current_choices, p_id: val},
                            inputs=[gr.State(pid), radio, etat_choix],
                            outputs=etat_choix
                        )
    generer_btn = gr.Button("Générer les réponses", variant="primary")
    with gr.Column() as generated_letters_output:
        @gr.render(inputs=[generated_letters_state])
        def render_generated_letters(letters):
            if not letters:
                gr.Markdown("Cliquez sur 'Générer les réponses' pour voir les lettres ici.")
                return

            for i, letter_content in enumerate(letters):
                gr.Textbox(label=f"Lettre {i+1}", value=letter_content, lines=10, interactive=False)

    def search_function(offre, k):
        res = matching(offre, k)
        detected_roles, detected_skills = detect_tender_features(offre)
        return res, {}, ", ".join(detected_roles) if detected_roles else "None", ", ".join(detected_skills) if detected_skills else "None"

    search_btn.click(
        fn=search_function,
        inputs=[offre_input, top_k_slider],
        outputs=[resultats_bruts, etat_choix, detected_roles_output, detected_skills_output]
    )

    def generer_toutes(results, choix, titre_offre):
        lettres_list = []
        for r in results:
            pid = r['profil']['id']
            decision = choix.get(pid)
            if decision:
                lettre = generer_reponse(r['profil'], decision, titre_offre)
                lettres_list.append(f"--- {r['profil']['nom']} ({decision}) ---\n{lettre}\n")
        return lettres_list

    generer_btn.click(
        fn=generer_toutes,
        inputs=[resultats_bruts, etat_choix, titre_offre_input],
        outputs=generated_letters_state
    )

demo.launch(share=True)

/tmp/ipython-input-220/723427926.py:1: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft(), title="SmartTender AI - Matching & Réponses") as demo:
/tmp/ipython-input-220/723427926.py:38: DeprecationWarning: The 'show_api' parameter in event listeners will be removed in Gradio 6.0. You will need to use the 'api_visibility' parameter instead. To replicate show_api=False, in Gradio 6.0, use api_visibility='undocumented'.
  @gr.render(inputs=[resultats_bruts, etat_choix])
/tmp/ipython-input-220/723427926.py:68: DeprecationWarning: The 'show_api' parameter in event listeners will be removed in Gradio 6.0. You will need to use the 'api_visibility' parameter instead. To replicate show_api=False, in Gradio 6.0, use api_visibility='undocumented'.
  @gr.render(inputs=[generated_letters_state])


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://56f029d95912eb4f07.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
